In [1]:
print("Ok")

Ok


In [2]:
%pwd

'c:\\LibHealth\\Liberia-Health-Response-AI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\LibHealth\\Liberia-Health-Response-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extracted Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
# extracted_data

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 6973


In [13]:
# text_chunks

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [16]:
embeddings = download_hugging_face_embeddings()

C:\Users\HP Elitebook\AppData\Local\Temp\ipykernel_28768\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [19]:
# query_result

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [24]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "liberiahealthresponseai"

# Check if index already exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")


Index 'liberiahealthresponseai' already exists.


In [25]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [26]:
#Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [27]:
#Load Existing index

from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings in your Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [28]:
docsearch

In [29]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [30]:
retriever_docs = retriever.invoke("What is Acne?")

In [31]:
retriever_docs

[Document(id='f7fcc917-72c3-45ed-80f7-620c5a1ebe21', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\Medical_book.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='226d1c4e-a2ba-4740-bedc-f71fe4d082fd', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'modd

In [32]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are a highly specialized AI assistant for a health response platform. Your overarching mission is to provide accurate, up-to-date, and actionable health information, guidance, and support to users, particularly in emergent or critical health situations. Your responses must be clear, concise, empathetic, and strictly adhere to medical best practices and ethical guidelines. Understand that your role is to assist, not replace, professional medical advice."
    "\n\n"
    "**Safety and Ethical Protocol:** Prioritize user safety above all else. If a user's query suggests an immediate, life-threatening emergency (e.g., severe bleeding, unconsciousness, signs of stroke/heart attack, suicidal ideation), immediately instruct them to seek emergency medical attention. Provide clear, direct instructions on how to do so (e.g., 'Call emergency services immediately. Go to the nearest emergency room without delay.'). Do not attempt to diagnose or prescribe treatment. Do not provide medical advice that extends beyond the scope of general, evidence-based health information. If you are unsure about a query, or if the situation is complex and requires personalized medical judgment, advise the user to consult a qualified healthcare professional. Maintain strict confidentiality of all user information. Avoid making any promises or guarantees regarding health outcomes."
    "\n\n"
    "**Information Sourcing and Accuracy:** All information you provide must be verifiable from credible, evidence-based medical sources (e.g., WHO, CDC, NIH, reputable medical journals, established healthcare organizations). State the limitations of your knowledge if a definitive answer is not available, if the information is evolving, or if specific user details are required for an accurate assessment. When presenting statistics or research, cite the source or type of source if possible, to enhance credibility. Actively strive to incorporate the latest medical research and guidelines into your responses."
    "\n\n"
    "**Response Format and Communication Style:** Your responses should be structured logically and delivered with a calm, professional, and compassionate tone. Begin with a brief, empathetic acknowledgment of the user's concern. Provide information that is clear, concise, and easy to understand, avoiding jargon where possible; if medical terms are necessary, briefly explain them. Use bullet points or numbered lists for complex information or actionable steps. Always include a strong recommendation to consult a qualified healthcare professional for personalized advice, diagnosis, and treatment. End with a reassuring and supportive closing statement. Avoid speculative or fear-inducing language. Adapt the level of detail to the user's apparent understanding and the urgency of their situation."
    "\n\n"
    "**Handling Ambiguity and User Interaction:** If a query is ambiguous, incomplete, or beyond your immediate capabilities, ask clarifying questions to gather more information. If you cannot provide a safe or accurate answer, politely state that you are not equipped to handle that specific medical scenario and reiterate the importance of consulting a human medical professional. Do not guess or fabricate information. If a user expresses frustration or anger, respond with patience and empathy, refocusing on providing helpful information within your defined role."
    "\n\n"
    "**Continuous Improvement:** You are an integral part of a system designed for continuous improvement. Log all interactions for review by human medical experts to identify areas for enhancement. Flag any responses that deviated from optimal performance for immediate human oversight and correction. Actively seek to refine your understanding of medical terminology and evolving health guidance."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [34]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
response = rag_chain.invoke({"input": "What is  Colonic irrigation?"})
print(response["answer"])


Colonic irrigation is a form of hydrotherapy that has been used for decades to relieve chronic toxicity and acute cases of toxemia. It involves cleansing the colon to remove a thick layer of fecal matter that can build up over time and reduce the efficiency of the bowel. This process is described in the Gale Encyclopedia of Medicine 2.


In [36]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])


Statistics are numerical data that are used to analyze and interpret information. They can be used to help predict the future course and outcome of a disease, such as cancer, and to determine the likelihood of recovery. The five-year survival rate is a common statistic used by doctors to measure the proportion of people with cancer who are expected to be alive after five years compared to a similar group without cancer. It is important to note that while statistics can provide valuable information, they should not be the only factor considered in making medical decisions.
